In [36]:
import numpy as np
import random
from itertools import product                   

In [263]:
def binomial(x):
    if isinstance(x, (list, tuple)):
        x = np.array(x, dtype=np.float32)
    return np.stack(
        [
            x[0]**2,
            x[0],
            x[0]*x[1],
            x[1],
            x[1]**2,
            np.ones(shape=x[0].shape)
        ], 0)

def new_layer(conn, layers, weights):
    last_layer = layers[-1]
    if conn:
        new_node = max(list(conn.keys()))+1
    else:
        new_node = max(layers[-1])+1
    new_layer = {n+new_node:x for n,x in enumerate(product(last_layer, repeat=2))}
    
    layers.append(list(new_layer.keys()))
    
    for n in layers[-1]:
        weights[n] = np.array([random.random() for _ in range(6)], dtype=np.float32)
    
    conn.update(new_layer)

def calculate(conn, layers, weights, values):
    """
    inputs: {input: [value list]} dict
    """
    values = {k: np.array(v, dtype=np.float32) for k, v in values.items()}
    for nl, layer in enumerate(layers[1:]):
        for nn, node in enumerate(layer):
            left, right = conn[node]
            values[node] = np.einsum("ij,i->j",binomial([values[left], values[right]]),weights[node])
    return values


def div_mx(conn, layers, weights, values, n_points):
    dn_dl = lambda w: lambda x, y: 2*w[0]*x+w[1]+w[2]*y
    dn_dr = lambda w: lambda x, y: 2*w[4]*y+w[3]+w[2]*x
    
    d = np.zeros((max(list(conn.keys()))+1,2,n_points))
    for layer in (layers[1:])[::-1]:
        for node in layer:
            left, right = conn[node]
            d[node][0] = np.array(list(map(dn_dl(weights[node]), values[left], values[right])), dtype=np.float32)
            d[node][1] = np.array(list(map(dn_dr(weights[node]), values[left], values[right])), dtype=np.float32)
    
    return d

def dn_dw(conn, layers, weights, layer_num, node_num, values, n_points):
    """
    """
    node = layers[layer_num][node_num]
    left, right = conn[node]
    dw = binomial((values[left], values[right]))
    
    dm = div_mx(conn, layers, weights, values, n_points)
    
    for layer in layers[layer_num:]:
        d = np.zeros(n_points)
        for pnode in layer:
            if node in conn[pnode]:
                pleft, pright = conn[pnode]
                if node == pleft:
                    d += dm[pnode][0]
                if node == pright:
                    d += dm[pnode][1]
                dw = np.einsum("ij,j->ij", dw, d)  
    return np.einsum("ij->i", dw)

def net_div(conn, layers, weights, values, root_error, n_points):
    nd = np.array([np.array([np.zeros(6) for n in l], dtype=np.float32) for l in layers], dtype=object)
    for nl, layer in enumerate(layers[1:]):
        for nn, node in enumerate(layer):
            nd[nl+1][nn] = dn_dw(conn, layers, weights, nl+1, nn, values, n_points)*root_error
    return nd

def regress(conn, layers, weights, inputs, true_values, keep=3, conv_thres=1, iter_thres=100):
    root_error_keep_best = np.zeros((keep))
    root_error_prev = np.ones((keep), dtype=np.float32)*conv_thres
    iteration = 0
    if isinstance(true_values, (list, tuple)):
        true_values = np.array(true_values, dtype=np.float32)
    n_points = true_values.shape[0]
    while (np.abs(root_error_keep_best - root_error_prev) >= conv_thres).all() and iteration < iter_thres:
        values = calculate(conn, layers, weights, inputs)
        out_layer_values = np.array([values[n] for n in layers[-1]], dtype=np.float32)
        iteration += 1
        root_error_prev = root_error_keep_best
        root_error = np.einsum("ij->i", out_layer_values - np.einsum("ij,j->ij", np.ones_like(out_layer_values), true_values))
        
        best_nodes, root_error_keep_best = list(zip(*sorted(zip(layers[-1], root_error), key=lambda x: x[1])[:keep]))
        root_error_keep_best = np.array(root_error_keep_best)
        root_error_norm = np.linalg.norm(root_error_keep_best)

        dw = [net_div(conn, layers, weights, values, err/root_error_norm, n_points) for err in root_error_keep_best]
        dw = np.sum(dw,0)
        for dln, dw_layer in enumerate(dw[1:]):
            for dnn, dw_node in enumerate(dw_layer):
                node = layers[dln+1][dnn]
                weights[node] += dw_node/np.linalg.norm(dw_node)
                
    return best_nodes, root_error_keep_best

def grow(inputs, 
         true_values,
         iter_thres = 100,
         goal = 1,
         keep = 3):
    """
    inputs: {input: [value list]} dict
    """
    model_inputs = tuple(inputs.keys())
    layers = [model_inputs]
    weights = {}
    connections = {}
    errors = np.array([goal])
    iteration = 0
    while (errors >= goal).all() and iteration < iter_thres:
        iteration += 1
        
        new_layer(connections, layers, weights)

        layers[-1], errors = regress(connections, layers, weights, inputs, true_values, keep)
    return connections, layers, weights, errors

In [ ]:
t_f = lambda x: x[1]**2
t_p = {i: [random.random() for _ in range(100)] for i in (0,1)}
t_y = list(map(t_f,zip(*t_p.values())))

print(t_p[1][10], t_y[10])

bino = lambda x,w: np.einsum("i->", w*binomial(x))
ass = np.array([1,1,1,1,1,1], dtype=np.float32)
o = grow(t_p, t_y)

o

0.8890232471015195 0.7903623338869293


In [254]:
np.sum(o,0)

array([array([[0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0.]], dtype=float32),
       array([[ -62.51611 ,  -90.281334,  -62.51611 ,  -90.281334,  -62.51611 ,
               -172.37709 ],
              [ -62.51611 ,  -90.281334,  -42.129406,  -83.31798 ,  -53.763855,
               -172.37709 ],
              [ -53.763855,  -83.31798 ,  -42.129406,  -90.281334,  -62.51611 ,
               -172.37709 ],
              [ -53.763855,  -83.31798 ,  -53.763855,  -83.31798 ,  -53.763855,
               -172.37709 ]], dtype=float32)                                   ],
      dtype=object)